<a href="https://colab.research.google.com/github/olu-sikeh/Oluomachi_DTSC3020_Fall2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [6]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")



Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [5]:
# Write your answer here

from pathlib import Path
import re
import csv

RAW_FILE = Path("contacts_raw.txt")
CLEAN_FILE = Path("contacts_clean.csv")
EMAIL_REGEX = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

def is_valid_email(email: str) -> bool:
  email = email.strip()
  return bool(EMAIL_REGEX.fullmatch(email))

def normalize_phone(raw: str) -> str:
  digits = re.sub(r"\D", "", raw)
  if len(digits) >= 10:
    return digits[-10:]
  else:
    return ""

def clean_contacts():
  try:
    lines = RAW_FILE.read_text(encoding="utf-8").splitlines()
  except FileNotFoundError:
    print(f"File not found: {RAW_FILE}.")
    return

  cleaned = []
  seen_emails = set()

  for line in lines:
    parts = [p.strip(' "').strip() for p in re.split(r"[<>,]", line) if p.strip()]
    if len(parts) < 2:
      continue #skip malformed rows

    #find email by regex
    email = next((p.strip() for p in parts if is_valid_email(p.strip())), "")
    if not email:
      continue

    email_key = email.casefold()
    if email_key in seen_emails:
      continue
    seen_emails.add(email_key)

    name = next((p for p in parts if not is_valid_email(p)), "") #non email token
    phone = normalize_phone(parts[-1]) # last token that looks like a number
    cleaned.append((name, email, phone))


# writing into CSV
with CLEAN_FILE.open("w", newline="", encoding="utf-8") as f:
   writer = csv.writer(f)
   writer.writerow(["name", "email", "phone"])
   writer.writerows(cleaned)

if __name__ == "__main__":
  clean_contacts()




## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [9]:
# Write your answer here

import unittest
from q1_crm_cleanup import is_valid_email, normalize_phone, clean_contacts
import q1_crm_cleanup as crm
from pathlib import Path

class TestCRMCleanup(unittest.TestCase):
  def test_is_valid_email(self):
    valid_emails = ["alice@example.com",
                    "bob.smith@domain.co", "mehdi_ay@example.org",
                    "user+tag@sub.domain.io"]
    for email in valid_emails:
      with self.subTest(email=email):
        self.assertTrue(is_valid_email(email))

def test_invalid_email(self):
  invalid_emails = ["bob[at]example.com", "missingatsymbol.com",
                    "name@domain", "@no-user.com"]
  for email in invalid_emails:
    with self.subTest(email=email):
      self.assertFalse(is_valid_email(email))

def test_phone_normalization(self):
  phones = {
      "+1 (469) 555-1234": "4695551234",
      "972-777-2121": "9727772121",
      "214 555 8888": "2145558888",
      "972.555.7777": "9725557777"
  }
  for raw, expected in phones.items():
    with self.subTest(phone=raw):
      self.assertEqual(normalize_phone(raw), expected)

def test_phone_too_short(self):
  self.assertEqual(normalize_phone("123-45"), "")

def test_clean_contacts(self):
  sample_data = ('Alice Johnson <alice@example.com> , +1 (469) 555-1234\n'
            'Bob Roberts <bob[at]example.com> , 972-555-777\n'
            'Sara M. , sara@mail.co , 214 555 8888\n'
            '"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\n'
            'Delaram <delaram@example.io>, +1-972-777-2121\n'
            'Nima <NIMA@example.io> , 972.777.2121\n'
            'duplicate <Alice@Example.com> , 469 555 1234')

 #write sample data into data file
  Path("contacts_raw.txt").write_text(sample_data, encoding="utf-8")
  crm.clean_contacts() #cleaning function

  #verify output
  output = Path("contacts_clean.csv").read_text(encoding="utf-8").splitlines()
  header, *rows = output
  self.assertEqual(header.strip(), "name,email,phone")
  self.assertEqual(len(rows), 5)

  self.assertIn(alice@example.com, rows[0])
  self.assertNotIn("Alice@Example.com", " ".join(rows[1:]))

if __name__ == "__main__":
  unittest.main()

ModuleNotFoundError: No module named 'q1_crm_cleanup'

## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
